In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data\artist_song\artist_song.csv
data\artist_song\artist_song.json
data\billboard\billboard_top_100.csv
data\input\songs-2.csv
data\input\top100.json
data\lyrics\artist_song_lyrics _bak.csv
data\lyrics\artist_song_lyrics.csv
data\lyrics\artist_song_lyrics_bak_old.csv
data\lyrics\short_lyrics.csv


# Import

In [2]:
top2020 = pd.read_json('data/input/top100.json')
top2020.head()

,id,rank,artist,song,rank_last_week,peak_rank,weeks_on_chart,date
0,1,1,Mariah Carey,All I Want For Christmas Is You,1 Last Week,1 Peak Rank,37 Weeks on Chart,2020-01-04
1,2,2,Brenda Lee,Rockin' Around The Christmas Tree,2 Last Week,2 Peak Rank,32 Weeks on Chart,2020-01-04
2,3,3,Bobby Helms,Jingle Bell Rock,9 Last Week,3 Peak Rank,30 Weeks on Chart,2020-01-04
3,4,4,Burl Ives,A Holly Jolly Christmas,6 Last Week,4 Peak Rank,15 Weeks on Chart,2020-01-04
4,5,5,Post Malone,Circles,3 Last Week,1 Peak Rank,17 Weeks on Chart,2020-01-04


In [3]:
top2019 = pd.read_csv('data/input/songs-2.csv')
top2019.head()

,Unnamed: 0,title,artist,peakPos,lastPos,weeks,isNew,rank,date,year
0,0,Mood,24kGoldn Featuring iann dior,1,1,15,False,1,2020-11-28,2020
1,1,Therefore I Am,Billie Eilish,2,94,2,False,2,2020-11-28,2020
2,2,Positions,Ariana Grande,1,2,4,False,3,2020-11-28,2020
3,3,I Hope,Gabby Barrett Featuring Charlie Puth,3,3,47,False,4,2020-11-28,2020
4,4,Laugh Now Cry Later,Drake Featuring Lil Durk,2,4,14,False,5,2020-11-28,2020


# Preparing 2019 Data
### Changing Cols

In [4]:
print(top2020.columns)
print(100*'_')
print(top2019.columns)


Index(['id', 'rank', 'artist', 'song', 'rank_last_week', 'peak_rank',
       'weeks_on_chart', 'date'],
      dtype='object')
____________________________________________________________________________________________________
Index(['Unnamed: 0', 'title', 'artist', 'peakPos', 'lastPos', 'weeks', 'isNew',
       'rank', 'date', 'year'],
      dtype='object')


In [5]:
top2019.rename(columns = {'title':'song', 'peakPos':'peak_rank', 'lastPos':'rank_last_week', 'weeks': 'weeks_on_chart'}, inplace = True)
top2019 = top2019.drop(['Unnamed: 0', 'isNew', 'year'], axis=1)
top2019.head()

,song,artist,peak_rank,rank_last_week,weeks_on_chart,rank,date
0,Mood,24kGoldn Featuring iann dior,1,1,15,1,2020-11-28
1,Therefore I Am,Billie Eilish,2,94,2,2,2020-11-28
2,Positions,Ariana Grande,1,2,4,3,2020-11-28
3,I Hope,Gabby Barrett Featuring Charlie Puth,3,3,47,4,2020-11-28
4,Laugh Now Cry Later,Drake Featuring Lil Durk,2,4,14,5,2020-11-28


### Checking DTypes

In [6]:
top2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   song            10000 non-null  object
 1   artist          10000 non-null  object
 2   peak_rank       10000 non-null  int64 
 3   rank_last_week  10000 non-null  int64 
 4   weeks_on_chart  10000 non-null  int64 
 5   rank            10000 non-null  int64 
 6   date            10000 non-null  object
dtypes: int64(4), object(3)
memory usage: 547.0+ KB


### Converting object to datetime

In [7]:
top2019['date'] = pd.to_datetime(top2019['date'], format='%Y-%m-%d')
top2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   song            10000 non-null  object        
 1   artist          10000 non-null  object        
 2   peak_rank       10000 non-null  int64         
 3   rank_last_week  10000 non-null  int64         
 4   weeks_on_chart  10000 non-null  int64         
 5   rank            10000 non-null  int64         
 6   date            10000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 547.0+ KB


In [8]:
top2019.head()

,song,artist,peak_rank,rank_last_week,weeks_on_chart,rank,date
0,Mood,24kGoldn Featuring iann dior,1,1,15,1,2020-11-28
1,Therefore I Am,Billie Eilish,2,94,2,2,2020-11-28
2,Positions,Ariana Grande,1,2,4,3,2020-11-28
3,I Hope,Gabby Barrett Featuring Charlie Puth,3,3,47,4,2020-11-28
4,Laugh Now Cry Later,Drake Featuring Lil Durk,2,4,14,5,2020-11-28


### Getting 2019 Charts

In [9]:
top2019 = top2019.loc[(top2019['date'] > '2018-12-31') & (top2019['date'] < '2020-01-04')]
print(top2019.shape)
print(top2019['date'].min())
print(top2019['date'].max())

(5200, 7)
2019-01-05 00:00:00
2019-12-28 00:00:00


# Preparing 2020 Data

### Checking Shape

In [10]:
top2020 = top2020.drop(['id'], axis=1)
print(top2020.shape)
print(top2020['date'].min())
print(top2020['date'].max())

(5300, 7)
2020-01-04 00:00:00
2021-01-02 00:00:00


In [11]:
top2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5300 entries, 0 to 5299
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   rank            5300 non-null   int64         
 1   artist          5300 non-null   object        
 2   song            5300 non-null   object        
 3   rank_last_week  5300 non-null   object        
 4   peak_rank       5300 non-null   object        
 5   weeks_on_chart  5300 non-null   object        
 6   date            5300 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 290.0+ KB


In [12]:
top2020['rank_last_week'] = top2020['rank_last_week'].apply(lambda rank_last_week: rank_last_week[:-10]).replace('-', 0)
top2020['peak_rank'] = top2020['peak_rank'].apply(lambda peak_rank: peak_rank[:-10]).replace('-', 0)
top2020['weeks_on_chart'] = top2020['weeks_on_chart'].apply(lambda weeks_on_chart: weeks_on_chart[:-15]).replace('-', 0)

for col in ['rank_last_week', 'peak_rank', 'weeks_on_chart']:
    top2020[col] = pd.to_numeric(top2020[col], errors='coerce')

# Merge

In [13]:
billboard_top_100 = top2019.merge(top2020, how='outer')
billboard_top_100.shape

(10500, 7)

In [14]:
billboard_top_100.sort_values(by=['date', 'rank'], inplace=True)
billboard_top_100.reset_index()
billboard_top_100.tail()

,song,artist,peak_rank,rank_last_week,weeks_on_chart,rank,date
10495,Good Time,Niko Moon,71,79,11,96,2021-01-02
10496,Throat Baby (Go Baby),BRS Kash,69,81,9,97,2021-01-02
10497,Errbody,Lil Baby,41,41,2,98,2021-01-02
10498,Favorite Time Of Year,Carrie Underwood,80,80,3,99,2021-01-02
10499,Beautiful Trip,Kid Cudi,100,0,1,100,2021-01-02


In [15]:
billboard_top_100.to_csv('data/billboard/billboard_top_100.csv')